In [1]:
import json
import matplotlib.pyplot as plt


# 读取数据
with open("../xfdata/dataset/job_list.json", "r", encoding="utf-8") as f:
    job_list = json.load(f)
with open("../xfdata/dataset/train.json", "r", encoding="utf-8") as f:
    train = json.load(f)
with open("../xfdata/dataset/test.json", "r", encoding="utf-8") as f:
    test = json.load(f)
    
# 1、教育经历
with open("../user_data/profileEduExps_sentences.json", "r", encoding="utf-8") as f:
    profileEduExps_sentences = json.load(f)

# 2、社会经历
with open("../user_data/profileSocialExps_sentences.json", "r", encoding="utf-8") as f:
    profileSocialExps_sentences = json.load(f)

# 3、项目经历
with open("../user_data/profileProjectExps_sentences.json", "r", encoding="utf-8") as f:
    profileProjectExps_sentences = json.load(f)

# 4、工作经历
with open("../user_data/profileWorkExps_sentences.json", "r", encoding="utf-8") as f:
    profileWorkExps_sentences = json.load(f)

# 5、技能
with open("../user_data/profileSkills_sentences.json", "r", encoding="utf-8") as f:
    profileSkills_sentences = json.load(f)

# 6、荣誉
with open("../user_data/profileAwards_sentences.json", "r", encoding="utf-8") as f:
    profileAwards_sentences = json.load(f)

# 7、求职意愿
with open("../user_data/profileDesire_sentences.json", "r", encoding="utf-8") as f:
    profileDesire_sentences = json.load(f)

# 8、语言
with open("../user_data/profileLanguage_sentences.json", "r", encoding="utf-8") as f:
    profileLanguage_sentences = json.load(f)

# 职位
with open("../user_data/job_sentences.json", "r", encoding="utf-8") as f:
    job_sentences = json.load(f)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch


# 获取标签
labels = [i["positionID"] for i in train]
labelencoder = LabelEncoder()
labels = labelencoder.fit_transform(labels)
print(len(np.unique(labels)))

51


In [3]:
sentences = []
# 调整位置
for s1, s2, s3, s4, s5, s6, s7, s8 in zip(profileEduExps_sentences, profileWorkExps_sentences, profileProjectExps_sentences, \
profileSocialExps_sentences, profileSkills_sentences, profileAwards_sentences, profileDesire_sentences, \
profileLanguage_sentences):
    sentences.append(s1 + s2 + s3 + s4 + s5 + s6 + s7 + s8)

train_sentences, test_sentences = sentences[: len(train)], sentences[len(train): ]
print(len(train_sentences), len(test_sentences))
train_idx = np.load("../user_data/train_idx.npy")
valid_idx = np.load("../user_data/valid_idx.npy")
train_sentences, valid_sentences = [train_sentences[idx] for idx in train_idx], [train_sentences[idx] for idx in valid_idx]
train_labels, valid_labels = [labels[idx] for idx in train_idx], [labels[idx] for idx in valid_idx]
print(len(train_sentences), len(train_labels))
print(len(valid_sentences), len(valid_labels))

20000 6500
16000 16000
4000 4000


# NSP

In [4]:
from transformers import BertForNextSentencePrediction, BertTokenizer, BertConfig

# 加载现有的MLM预训练模型权重路径
path = "../user_data/output/checkpoint-132800"
# 加载配置
config = BertConfig.from_pretrained(path)
# 实例化 NSP 任务的模型
model = BertForNextSentencePrediction.from_pretrained(path, config=config)
# 加载同样的 tokenizer
tokenizer = BertTokenizer.from_pretrained(path)

Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at ../user_data/output/checkpoint-132800 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from datasets import load_from_disk


dataset_for_nsp = load_from_disk("../user_data/dataset_for_nsp")

In [6]:
from tqdm import tqdm 
from datasets import Dataset


# 获取每行中最大的三个值及其索引
topk_values, topk_indices = torch.topk(dataset_for_nsp["train"]["logits"], k=10, dim=1)
# 通过循环构建数据集
input_ids_train = []
token_type_ids_train = []
atttention_mask_train = []
labels_train = []
for i in tqdm(range(16000)):
    topk_indice = topk_indices[i]
    label = train_labels[i]
    for j in set(topk_indice.tolist() + [label]):
        resume = train_sentences[i]
        job = job_sentences[j]
        input_ids, token_type_ids, attention_mask = tokenizer.encode_plus(
            resume, 
            job,
            padding="max_length",
            max_length=1024,
            truncation="only_first",
            return_tensors="pt"
        ).values()
        input_ids_train.append(input_ids)
        token_type_ids_train.append(token_type_ids)
        atttention_mask_train.append(attention_mask)
        if j == label:
            labels_train.append(1)
        else:
            labels_train.append(0)

input_ids_train = torch.cat(input_ids_train)
token_type_ids_train = torch.cat(token_type_ids_train)
atttention_mask_train = torch.cat(atttention_mask_train)

train_encoded_nsp = Dataset.from_dict({
    "input_ids": input_ids_train,
    "token_type_ids": token_type_ids_train,
    "attention_mask": atttention_mask_train,
    "labels": labels_train
})

train_encoded_nsp.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

100%|██████████| 16000/16000 [03:23<00:00, 78.63it/s]


In [7]:
# 获取每行中最大的三个值及其索引
topk_values, topk_indices = torch.topk(dataset_for_nsp["valid"]["logits"], k=10, dim=1)
# 通过循环构建数据集
input_ids_valid = []
token_type_ids_valid = []
atttention_mask_valid = []
labels_valid = []
for i in tqdm(range(4000)):
    topk_indice = topk_indices[i]
    label = valid_labels[i]
    for j in set(topk_indice.tolist() + [label]):
        resume = valid_sentences[i]
        job = job_sentences[j]
        input_ids, token_type_ids, attention_mask = tokenizer.encode_plus(
            resume, 
            job,
            padding="max_length",
            max_length=1024,
            truncation="only_first",
            return_tensors="pt"
        ).values()
        input_ids_valid.append(input_ids)
        token_type_ids_valid.append(token_type_ids)
        atttention_mask_valid.append(attention_mask)
        if j == label:
            labels_valid.append(1)
        else:
            labels_valid.append(0)

input_ids_valid = torch.cat(input_ids_valid)
token_type_ids_valid = torch.cat(token_type_ids_valid)
atttention_mask_valid = torch.cat(atttention_mask_valid)

valid_encoded_nsp = Dataset.from_dict({
    "input_ids": input_ids_valid,
    "token_type_ids": token_type_ids_valid,
    "attention_mask": atttention_mask_valid,
    "labels": labels_valid
})

valid_encoded_nsp.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

100%|██████████| 4000/4000 [00:49<00:00, 80.23it/s]


In [8]:
# 获取每行中最大的三个值及其索引
topk_values, topk_indices = torch.topk(dataset_for_nsp["test"]["logits"], k=10, dim=1)
# 通过循环构建数据集
input_ids_test = []
token_type_ids_test = []
atttention_mask_test = []
for i in tqdm(range(6500)):
    topk_indice = topk_indices[i]
    for j in set(topk_indice.tolist()):
        resume = test_sentences[i]
        job = job_sentences[j]
        input_ids, token_type_ids, attention_mask = tokenizer.encode_plus(
            resume, 
            job,
            padding="max_length",
            max_length=1024,
            truncation="only_first",
            return_tensors="pt"
        ).values()
        input_ids_test.append(input_ids)
        token_type_ids_test.append(token_type_ids)
        atttention_mask_test.append(attention_mask)
        
input_ids_test = torch.cat(input_ids_test)
token_type_ids_test = torch.cat(token_type_ids_test)
atttention_mask_test = torch.cat(atttention_mask_test)

test_encoded_nsp = Dataset.from_dict({
    "input_ids": input_ids_test,
    "token_type_ids": token_type_ids_test,
    "attention_mask": atttention_mask_test,
})

test_encoded_nsp.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask"])

100%|██████████| 6500/6500 [01:21<00:00, 79.71it/s]


In [9]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="../user_data/nsp_output",
    evaluation_strategy="epoch",  # 每个epoch后进行验证
    per_device_train_batch_size=64,  # 可以根据显存调整
    per_device_eval_batch_size=64,
    num_train_epochs=10,  # 微调的轮数
    weight_decay=0.01,
    logging_dir='../user_data/nsp_log',  # 保存日志文件的位置
    save_strategy="epoch",  # 每个epoch保存一次模型
    load_best_model_at_end=True,
    greater_is_better=False,  
)

2024-09-16 19:57:02.432178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 19:57:02.447592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-16 19:57:02.452238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-16 19:57:02.465562: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-16 19:57:03.098533: W tensorflow/compiler/tf2

In [10]:
from sklearn.utils import compute_class_weight
import torch


# 计算类别权重
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_encoded_nsp["labels"].numpy()), y=train_encoded_nsp["labels"].numpy())
# 转换为tensor格式以便后续使用
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

# 定义带权重的损失函数
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# 自定义Trainer
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to("cuda")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded_nsp,
    eval_dataset=valid_encoded_nsp,
    tokenizer=tokenizer
)
trainer.train()

Epoch,Training Loss,Validation Loss


In [13]:
# 获取每行中最大的三个值及其索引
topk_values, topk_indices = torch.topk(dataset_for_nsp["valid"]["logits"], k=10, dim=1)
# 通过循环构建数据集
input_ids_valid = []
token_type_ids_valid = []
atttention_mask_valid = []
labels_valid = []
for i in tqdm(range(4000)):
    topk_indice = topk_indices[i]
    label = valid_labels[i]
    for j in set(topk_indice.tolist()):
        resume = valid_sentences[i]
        job = job_sentences[j]
        input_ids, token_type_ids, attention_mask = tokenizer.encode_plus(
            resume, 
            job,
            padding="max_length",
            max_length=1024,
            truncation="only_first",
            return_tensors="pt"
        ).values()
        input_ids_valid.append(input_ids)
        token_type_ids_valid.append(token_type_ids)
        atttention_mask_valid.append(attention_mask)
        if j == label:
            labels_valid.append(1)
        else:
            labels_valid.append(0)

input_ids_valid = torch.cat(input_ids_valid)
token_type_ids_valid = torch.cat(token_type_ids_valid)
atttention_mask_valid = torch.cat(atttention_mask_valid)

valid_encoded_nsp = Dataset.from_dict({
    "input_ids": input_ids_valid,
    "token_type_ids": token_type_ids_valid,
    "attention_mask": atttention_mask_valid,
    "labels": labels_valid
})

valid_encoded_nsp.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
result_valid = trainer.predict(valid_encoded_nsp)

100%|██████████| 4000/4000 [00:49<00:00, 81.42it/s]


In [23]:
from torch import nn
from sklearn.metrics import f1_score

# 获取每行中最大的三个值及其索引
topk_values, topk_indices = torch.topk(dataset_for_nsp["valid"]["logits"], k=10, dim=1)
start = 0
valid_logits = result_valid.predictions[:, 1]
pred_valid = []
count = 0
for i in range(4000):
    topk_indice = topk_indices[i]
    label = valid_labels[i]
    choices = list(set(topk_indice.tolist()))
    logits_choices = valid_logits[: len(choices)]
    if nn.Softmax(dim=1)(topk_values[[i]])[0][0] < 0.25:
        count += 1
        choice = choices[logits_choices.argmax()]
    else: 
        choice = topk_indice[0]
    pred_valid.append(choice)
    valid_logits = valid_logits[len(choices): ]
print(count)
f1_score(y_pred=pred_valid, y_true=valid_labels, average="macro")

333


0.37697416956310054

In [ ]:
0.375266757877344

In [265]:
result = trainer.predict(test_encoded_nsp)

In [277]:
# 获取每行中最大的三个值及其索引
topk_values, topk_indices = torch.topk(dataset_nsp["test"]["logits"], k=3, dim=1)
start = 0
test_logits = result.predictions[:, 1]
pred_test = []
for i in range(6500):
    topk_indice = topk_indices[i]
    choices = topk_indice
    logits_choices = test_logits[: len(choices)]
    if nn.Softmax(dim=1)(topk_values[[i]])[0][0] < 0.6:
        choice = choices[logits_choices.argmax()]
    else: 
        choice = topk_indice[0]
    pred_test.append(choice.item())
    test_logits = test_logits[len(choices): ]

In [282]:
df_submit = pd.read_csv("../xfdata/example.csv")
df_submit["positionID"] = labelencoder.inverse_transform(pred_test)
df_submit.to_csv("../prediction_result/cls_nsp.csv", index=False)